<a href="https://colab.research.google.com/github/SCCSMARTCODE/Deep-Learning-03-LLM-FineTuning/blob/main/gemma-2b-qlora-english-quotes/fine_tuning_gemma_2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall transformers peft trl bitsandbytes accelerate -y
!pip cache purge

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
Files removed: 0


In [2]:
!pip install --upgrade pip

!pip install transformers peft bitsandbytes accelerate trl safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 176.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 168.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB

In [16]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import PeftModel, LoraConfig
from datasets import load_dataset
from trl import SFTTrainer
from dataclasses import dataclass, field
from google.colab import userdata

In [4]:
@dataclass
class ModelConfig:
    model_name: str = "google/gemma-2-2b"

In [5]:
model  = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=ModelConfig.model_name,
    attn_implementation='eager',
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    ),
    device_map="auto",
    use_auth_token=userdata.get("FIRST_GEMMA_2B_FINETUNING_HF_TOKEN")
)

model.config.use_cache = False

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
from google.colab import userdata

tokenizer = AutoTokenizer.from_pretrained(
    ModelConfig.model_name,
    trust_remote_code=True,
    use_auth_token=userdata.get("FIRST_GEMMA_2B_FINETUNING_HF_TOKEN")
    )


tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
dataset = load_dataset("Abirate/english_quotes", split="train[:2000]")

def to_metadata_example(example):
    quote = example["quote"].strip()
    author = example["author"]
    tags = example.get("tags", [])

    # Join tags as comma-separated string
    tags_str = ", ".join(tags)

    chatml = (
        "<start_of_turn>user\n"
        f"{quote}\n"
        "<end_of_turn>\n"
        "<start_of_turn>model\n"
        f"Author: {author}\n"
        f"Tags: {tags_str}\n"
        "<end_of_turn>"
    )

    return {"text": chatml}

meta_dataset = dataset.map(to_metadata_example, remove_columns=dataset.column_names)

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
meta_dataset[0]

{'text': '<start_of_turn>user\n“Be yourself; everyone else is already taken.”\n<end_of_turn>\n<start_of_turn>model\nAuthor: Oscar Wilde\nTags: be-yourself, gilbert-perreira, honesty, inspirational, misattributed-oscar-wilde, quote-investigator\n<end_of_turn>'}

In [10]:
training_args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    optim="paged_adamw_8bit",
    save_steps=500,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,             # Only if your GPU supports it (e.g., T4)
    max_grad_norm=1.0,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=meta_dataset,
    peft_config=peft_config,
    args=training_args,
    processing_class=tokenizer,
    # label_names=["labels"]
)

Converting train dataset to ChatML:   0%|          | 0/2000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
trainer.train()

Step,Training Loss
10,3.428500
20,2.832800
30,2.127600
40,2.038000
50,1.772700
60,1.672200
70,1.813400
80,1.599500
90,1.443700
100,1.680900


Step,Training Loss
10,3.428500
20,2.832800
30,2.127600
40,2.038000
50,1.772700
60,1.672200
70,1.813400
80,1.599500
90,1.443700
100,1.680900


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-682cef8e-6526d9b828d3ab6e696a99d6;84b45f6e-47ba-49f8-a413-61c269f0d8b8)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b/resolve/main/config.json.
Access to model google/gemma-2-2b is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2-2b.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in google/gemma-2-2b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=250, training_loss=1.659619255065918, metrics={'train_runtime': 1343.6349, 'train_samples_per_second': 2.977, 'train_steps_per_second': 0.186, 'total_flos': 3331421318320128.0, 'train_loss': 1.659619255065918})

In [12]:
trainer.model.save_pretrained("finetuned_gemma2b_lora")
tokenizer.save_pretrained("finetuned_gemma2b_lora")

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-682cf0ef-2867e5f615ee21bf02799e98;89dec97f-1bd8-48cf-9403-4180792c3090)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b/resolve/main/config.json.
Access to model google/gemma-2-2b is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2-2b.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in google/gemma-2-2b - will assume that the vocabulary was not modified.
  warnings.warn(


('finetuned_gemma2b_lora/tokenizer_config.json',
 'finetuned_gemma2b_lora/special_tokens_map.json',
 'finetuned_gemma2b_lora/tokenizer.model',
 'finetuned_gemma2b_lora/added_tokens.json',
 'finetuned_gemma2b_lora/tokenizer.json')

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `FIRST_GEMMA_2B_FINETUNING` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have t

In [25]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.int16
)

Device set to use cuda:0


In [30]:


input_text = "<start_of_turn>user\n“Be yourself; everyone else is already taken.”\n<end_of_turn>\n<start_of_turn>model\n"
inputs = tokenizer(input_text, return_tensors="pt")
inputs = {k: v.to('cuda') for k, v in inputs.items()}
inputs['attention_mask'] = inputs['attention_mask'].long()

from torch.amp import autocast

with autocast("cuda", dtype=torch.float16):
    outputs = model.generate(**inputs, max_new_tokens=32)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
“Be yourself; everyone else is already taken.”

model
Author: Oscar Wilde
Tags: inspirational, inspirational-quotes, inspirational-wisdom, inspirational-wisdom-oscar-wilde



In [15]:
model.push_to_hub("SCCSMARTCODE/finetuned-gemma2b-lora")
tokenizer.push_to_hub("SCCSMARTCODE/finetuned-gemma2b-lora")

model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/SCCSMARTCODE/finetuned-gemma2b-lora/commit/a8c72263cfd5a304482bb15175a539a901049dfb', commit_message='Upload tokenizer', commit_description='', oid='a8c72263cfd5a304482bb15175a539a901049dfb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SCCSMARTCODE/finetuned-gemma2b-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='SCCSMARTCODE/finetuned-gemma2b-lora'), pr_revision=None, pr_num=None)